# REMDのOpenMM実装
- ChatGPTを使用
- 一定時間ごとに、レプリカ間の座標を交換する(温度はそのまま)
- 速度は交換しない

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np
from tqdm.notebook import tqdm
import random
import os
from datetime import datetime
from math import ceil

In [2]:
# 現在の日付と時刻を取得
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# ディレクトリ名を作成
output_dir = f"./output/output_{current_time}"
# ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)
print(f"ディレクトリ '{output_dir}' を作成しました！")

ディレクトリ './output/output_2024-12-09_15-31-28' を作成しました！


In [14]:
# パラメータ設定
params = {}
params['n_replicas'] = 8  # レプリカ数
replicas = [str(i) for i in np.arange(params['n_replicas'])]

params['temperature_min'] = 300 # K
params['temperature_max'] = 500 # K
params['temperatures'] = np.linspace(params['temperature_min'], params['temperature_max'], params['n_replicas'])  # 温度 (Kelvin)
params['temperatures'] = params['temperatures'].tolist()

params['dt'] = 0.002 # タイムステップ(ps)
params['n_steps'] = 200 * int(ceil(1e0 / params['dt'])) # シミュレーション総ステップ数(1us) 1e6
params['n_steps_exchange'] = 2 * int(ceil(1e0 / params['dt'])) # 交換を試みる間隔(1ns) 1e3
params['n_steps_save'] = 1 * int(ceil(1e0 / params['dt'])) # 保存間隔(100ps) 100
params['n_steps_equil'] = 50 * int(ceil(1e0 / params['dt'])) # 平衡化(10ns) 20000

params['n_times_exchange'] = params['n_steps'] // params['n_steps_exchange']

params['nonbondedCutoff'] = 1.0 # nm
params['friction'] = 1.0 # /ps
params['restraint_force'] = 10 # kcal/mol/A^2

params['pdb_path'] = './structures/ala2_solvated.pdb'
params['ff'] = ['amber99sbildn.xml', 'tip3p.xml']

for key in params.keys():
    print(f'{key}: {params[key]}')

n_replicas: 8
temperature_min: 300
temperature_max: 500
temperatures: [300.0, 328.57142857142856, 357.14285714285717, 385.7142857142857, 414.2857142857143, 442.8571428571429, 471.42857142857144, 500.0]
dt: 0.002
n_steps: 100000
n_steps_exchange: 1000
n_steps_save: 500
n_steps_equil: 25000
n_times_exchange: 100
nonbondedCutoff: 1.0
friction: 1.0
restraint_force: 10
pdb_path: ./structures/ala2_solvated.pdb
ff: ['amber99sbildn.xml', 'tip3p.xml']


In [15]:
# パラメータ保存
import json
filepath = f"{output_dir}/params.json"
with open(filepath, mode="wt", encoding="utf-8") as f:
	json.dump(params, f, ensure_ascii=False, indent=2)

## レプリカ交換関数の定義

In [5]:
# # エネルギー交換関数
# # アボガドロ定数で割るバージョン
# # kJ/moleに直すためとはいえ、アボガドロ定数で割るのは間違っているので没
# def attempt_exchange(replica1, replica2):
#     E1 = simulations[replica1].context.getState(getEnergy=True).getPotentialEnergy()
#     E2 = simulations[replica2].context.getState(getEnergy=True).getPotentialEnergy()
#     beta1 = 1 / (BOLTZMANN_CONSTANT_kB * params['temperatures'][replica1] * kelvin)
#     beta2 = 1 / (BOLTZMANN_CONSTANT_kB * params['temperatures'][replica2] * kelvin)
#     delta = (beta2 - beta1) * (E1 - E2) / AVOGADRO_CONSTANT_NA
#     print(delta)
#     if delta < 0 or random.uniform(0, 1) < np.exp(-delta):
#         # 交換を行う
#         # print(f"Exchange accepted between replica {replica1} and {replica2}")
#         # temp1 = temperatures[replica1]
#         # temperatures[replica1] = temperatures[replica2]
#         # temperatures[replica2] = temp1
#         rep1 = replicas[replica1]
#         replicas[replica1] = replicas[replica2]
#         replicas[replica2] = rep1
#         # Write log
#         with open(f'{output_dir}/acceptance.csv', 'a') as f:
#             # f.write(','.join(replicas)+'\n')
#             f.write(f"{replica1},{replica2},accept\n")
            
#         positions1 = simulations[replica1].context.getState(getPositions=True).getPositions()
#         positions2 = simulations[replica2].context.getState(getPositions=True).getPositions()
#         simulations[replica1].context.setPositions(positions2)
#         simulations[replica2].context.setPositions(positions1)
#     else:
#         # print(f"Exchange rejected between replica {replica1} and {replica2}")
#         # Write log
#         with open(f'{output_dir}/acceptance.csv', 'a') as f:
#             # f.write(','.join(replicas)+'\n')
#             f.write(f"{replica1},{replica2},reject\n")
#         pass

In [6]:
# エネルギー交換関数
# 全部value_in_unitする
def attempt_exchange(replica1, replica2):
    E1 = simulations[replica1].context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kilojoule_per_mole)
    E2 = simulations[replica2].context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kilojoule_per_mole)
    kB = MOLAR_GAS_CONSTANT_R.value_in_unit(kilojoule_per_mole/kelvin)
    beta1 = 1 / (kB * params['temperatures'][replica1])
    beta2 = 1 / (kB * params['temperatures'][replica2])
    delta = (beta2 - beta1) * (E1 - E2)
    print(delta)
    if delta < 0 or random.uniform(0, 1) < np.exp(-delta):
        # 交換を行う
        # print(f"Exchange accepted between replica {replica1} and {replica2}")
        # temp1 = temperatures[replica1]
        # temperatures[replica1] = temperatures[replica2]
        # temperatures[replica2] = temp1
        rep1 = replicas[replica1]
        replicas[replica1] = replicas[replica2]
        replicas[replica2] = rep1
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{replica1},{replica2},accept\n")
            
        positions1 = simulations[replica1].context.getState(getPositions=True).getPositions()
        positions2 = simulations[replica2].context.getState(getPositions=True).getPositions()
        simulations[replica1].context.setPositions(positions2)
        simulations[replica2].context.setPositions(positions1)
    else:
        # print(f"Exchange rejected between replica {replica1} and {replica2}")
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{replica1},{replica2},reject\n")
        pass

## システムの作成

In [7]:
pdb = app.PDBFile(params['pdb_path'])
forcefield = app.ForceField(*params['ff'])

In [8]:
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
print('System created...')

# レプリカごとに異なるシミュレーションをセットアップ
integrators = []
simulations = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction']/picosecond,    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    integrators.append(integrator)
    simulation = app.Simulation(pdb.topology, system, integrator)
    # simulation.context.setPositions(pdb.positions)
    # simulation.context.setVelocitiesToTemperature(temp*kelvin)
    simulations.append(simulation)

System created...


## 平衡化

In [9]:
%%time
system_equil = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
# Positional restraints for all heavy-atoms for equilibration
pos_res = CustomExternalForce("k*periodicdistance(x, y, z, x0, y0, z0)^2;")
pos_res.addPerParticleParameter("k")
pos_res.addPerParticleParameter("x0")
pos_res.addPerParticleParameter("y0")
pos_res.addPerParticleParameter("z0")

for ai, atom in enumerate(pdb.topology.atoms()):
    if atom.name == 'CA':
        x = pdb.positions[ai][0].value_in_unit(nanometers)
        y = pdb.positions[ai][1].value_in_unit(nanometers)
        z = pdb.positions[ai][2].value_in_unit(nanometers)
        pos_res.addParticle(ai, [params['restraint_force']*kilocalories_per_mole/(angstrom**2), x, y, z])

system_equil.addForce(pos_res)
print('System_equil created...')

integrators_equil = []
simulations_equil = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction'],    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    # integrators_equil.append(integrator)
    simulation_equil = app.Simulation(pdb.topology, system_equil, integrator)
    simulation_equil.context.setPositions(pdb.positions)
    simulation_equil.minimizeEnergy()
    simulation_equil.context.setVelocitiesToTemperature(temp*kelvin)
    print("Equilibrating for", params['n_steps_equil'], "steps")
    simulation_equil.step(params['n_steps_equil'])
    # simulations_equil.append(simulation)
    simulations[i].context.setPositions(
        simulation_equil.context.getState(getPositions=True ).getPositions()
    )
    simulations[i].context.setVelocities(
        simulation_equil.context.getState(getVelocities=True).getVelocities()
    )

System_equil created...
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
CPU times: user 26 s, sys: 3.46 s, total: 29.4 s
Wall time: 29.9 s


## レポーターの追加

In [10]:
# append reporters
print('Adding Reporters...')
dcd_reporters = [app.DCDReporter(file=f'{output_dir}/replica_{i}.dcd', reportInterval=params['n_steps_save']) 
                 for i, temp in enumerate(params['temperatures'])]
state_data_reporters = [app.StateDataReporter(file=f'{output_dir}/replica_{i}.log', 
                                              reportInterval=params['n_steps_save'], 
                                              totalSteps=params['n_steps'], 
                                              step=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              speed=True, 
                                              progress=True, 
                                              elapsedTime=True) 
                        for i, temp in enumerate(params['temperatures'])]

# 各レプリカにreporterを追加
for i, temp in enumerate(params['temperatures']):
    simulations[i].reporters.append(dcd_reporters[i])
    simulations[i].reporters.append(state_data_reporters[i])

Adding Reporters...


## Production
- 隣接ペアの組み合わせを1回毎に変えるようにしたい
- accept/rejectのログを記録したい

In [11]:
for i in range(1, 8-1, 2):
    print(i, i+1)

1 2
3 4
5 6


In [12]:
%%time
# シミュレーションループ
print('Production...')
is_even_step = True # 交換ペアを交互に変えるための変数
for step in tqdm(range(0, params['n_steps'], params['n_steps_exchange']), leave=False):
    # print(f'*** Step {step} ***"')
    for sim in simulations:
        sim.step(params['n_steps_exchange'])  # 各レプリカで実行

    # レプリカ間での交換試行
    if is_even_step:
        for i in range(0, params['n_replicas'], 2):
            attempt_exchange(i, i + 1)
    else:
        for i in range(1, params['n_replicas']-1, 2):
            attempt_exchange(i, i + 1)
            
    is_even_step = not is_even_step
        
    with open(f'{output_dir}/replicas.csv', 'a') as f:
            f.write(','.join(replicas)+'\n')

Production...


  0%|          | 0/100 [00:00<?, ?it/s]

8.98587429142831
7.4748399058155055
5.4266082275501555
3.8983401033571488
4.660127020831922
6.7441713218811765
7.518329719651733
4.20385958269868
6.722115940258296
0.6752135833274262
3.6607305980474623
9.16787881978696
3.957395682746235
6.382828968909617
9.507201393296816
6.500218059241469
2.7749712622134433
5.564322072163854
8.472732402301917
3.3035708132428363
2.6044745444167656
2.582396171401049
4.148647425681758
4.09783211123744
3.2970354123572965
7.603410404807051
3.2876724416115963
4.507592714165086
3.1633114256020276
3.0600267649205413
2.3570935645815223
3.2978477347853894
9.953761369342576
13.547586155609732
5.418388322180833
12.19242337656696
4.584545640330814
8.888284923320949
2.006063061309224
9.720538199986827
6.522964528912643
1.8147145911028135
11.102075751637742
5.753918543185966
3.2207287310883785
3.9227463926464696
4.244348805147832
8.283299175235486
4.137778649265069
12.228391928524228
6.954805932642098
0.039266904355199306
0.40844057654683524
14.280222443393757
7.592

## 最終構造を保存

In [13]:
# シミュレーション結果を保存
for i, sim in enumerate(simulations):
    state = sim.context.getState(getPositions=True, getEnergy=True)
    with open(f"{output_dir}/output_replica_{i}.pdb", "w") as f:
        app.PDBFile.writeFile(pdb.topology, state.getPositions(), f)